#**Tugas Kelompok TK-03**

Nama Kelompok : Nougat

Nama Anggota  : 
1. Suryadi (1706071245)
2. Kukuh Nur Aji (2006494464)
3. Jelita Permatasari (2006547123)

#**Language Model**


In [ ]:
!pip install -U textblob 
!python -m textblob.download_corpora
!pip install -U pip
!pip install -U dill
!pip install -U nltk==3.4

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


**N-grams Language Models (N-grams LM)**

kita dapat menggunakan n-grams untuk menghasilkan model bahasa yang dapat memprediksi kata selanjutnya dari riwayat kata-kata yang digunakan sebelumnya.
Salah satu library python yang sangat membantu dalam pembuatan model bahasa yaitu NLTK (Natural Language Tool Kit).

(**Sumber**: Sebagian besar konten di notebook ini didasarkan pada Language Model Tutorial in NLTK Documentation by Ilia Kurenkov)

In [ ]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
import pandas as pd

Berikut dibawah ini adalah contoh text untuk implementasi bigrams dan n-gram.

In [ ]:
text = [['a', 'b', 'c'], ['a', 'c', 'd', 'c', 'e', 'f']]

In [ ]:
list(bigrams(text[0]))

[('a', 'b'), ('b', 'c')]

In [ ]:
list(ngrams(text[1], n=3))

[('a', 'c', 'd'), ('c', 'd', 'c'), ('d', 'c', 'e'), ('c', 'e', 'f')]

Dalam pembuatan model n-grams, data pelatihan untuk model harus dibersihkan dan dirapihkan terlebih dahulu karena akan berpengaruh terhadap hasil keluaran model bahasa n-grams. Salah satu cara untuk merapihkan data yaitu dengan menambahkan padding. padding berfungsi untuk menandakan simbol pada data pelatihan yang menyatakan awalan dan akhiran dari sebuah kalimat.
`<s>` = awalan kalimat
`</s>` = akhiran kalimat

In [ ]:
from nltk.util import pad_sequence
list(pad_sequence(text[0],
                  pad_left=True, left_pad_symbol="<s>",
                  pad_right=True, right_pad_symbol="</s>",
                  n=2)) # The n order of n-grams, if it's 2-grams, you pad once, 3-grams pad twice, etc. 

['<s>', 'a', 'b', 'c', '</s>']

In [ ]:
padded_sent = list(pad_sequence(text[0], pad_left=True, left_pad_symbol="<s>", 
                                pad_right=True, right_pad_symbol="</s>", n=2))
list(ngrams(padded_sent, n=2))

[('<s>', 'a'), ('a', 'b'), ('b', 'c'), ('c', '</s>')]

In [ ]:
list(pad_sequence(text[0],
                  pad_left=True, left_pad_symbol="<s>",
                  pad_right=True, right_pad_symbol="</s>",
                  n=3)) # The n order of n-grams, if it's 2-grams, you pad once, 3-grams pad twice, etc. 

['<s>', '<s>', 'a', 'b', 'c', '</s>', '</s>']

In [ ]:
padded_sent = list(pad_sequence(text[0], pad_left=True, left_pad_symbol="<s>", 
                                pad_right=True, right_pad_symbol="</s>", n=3))
list(ngrams(padded_sent, n=3))

[('<s>', '<s>', 'a'),
 ('<s>', 'a', 'b'),
 ('a', 'b', 'c'),
 ('b', 'c', '</s>'),
 ('c', '</s>', '</s>')]

In [ ]:
from nltk.lm.preprocessing import pad_both_ends
list(pad_both_ends(text[0], n=2))

['<s>', 'a', 'b', 'c', '</s>']

Penggabungan 2 step diatas yaitu pembuatan bigrams dan penambahan padding untuk mempersiapkan data pelatihan

In [ ]:
list(bigrams(pad_both_ends(text[0], n=2)))

[('<s>', 'a'), ('a', 'b'), ('b', 'c'), ('c', '</s>')]

Pembuatan model yang robust dapat dilakukan dengan melatih machine learning dengan banyak jenis data seperti unigrams,bigrams,trigrams,dst. NLTK menyediakan fitur evergrams untuk menghasilkan setiap jenis data n-gram

In [ ]:
from nltk.util import everygrams
padded_bigrams = list(pad_both_ends(text[0], n=2))
list(everygrams(padded_bigrams, max_len=2))

[('<s>',),
 ('a',),
 ('b',),
 ('c',),
 ('</s>',),
 ('<s>', 'a'),
 ('a', 'b'),
 ('b', 'c'),
 ('c', '</s>')]

Corpus yang dihasilkan dari proses n-grams dan padding kemudian dijadikan satu untuk mempermudah proses counting.

In [ ]:
from nltk.lm.preprocessing import flatten
list(flatten(pad_both_ends(sent, n=2) for sent in text))

['<s>', 'a', 'b', 'c', '</s>', '<s>', 'a', 'c', 'd', 'c', 'e', 'f', '</s>']

In [ ]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(2, text)

Diatas adalah contoh proses pembuatan data pelatihan NLP menggunakan contoh data. Dibawah ini merupakan proses pembuatan token dari dataset teks bahasa dan melakukan proses pembersihan dan perapihan seperti step diatas.

In [ ]:
try: # Use the default NLTK tokenizer.
    from nltk import word_tokenize, sent_tokenize 
    # Testing whether it works. 
    # Sometimes it doesn't work on some machines because of setup issues.
    word_tokenize(sent_tokenize("This is a foobar sentence. Yes it is.")[0])
except: # Use a naive sentence tokenizer and toktok.
    import re
    from nltk.tokenize import ToktokTokenizer
    # See https://stackoverflow.com/a/25736515/610569
    sent_tokenize = lambda x: re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', x)
    # Use the toktok tokenizer that requires no dependencies.
    toktok = ToktokTokenizer()
    word_tokenize = word_tokenize = toktok.tokenize

In [ ]:
import os
import requests
import io #codecs


# Text version of https://kilgarriff.co.uk/Publications/2005-K-lineer.pdf
if os.path.isfile('language-never-random.txt'):
    with io.open('language-never-random.txt', encoding='utf8') as fin:
        text = fin.read()
else:
    url = "https://gist.githubusercontent.com/alvations/53b01e4076573fea47c6057120bb017a/raw/b01ff96a5f76848450e648f35da6497ca9454e4a/language-never-random.txt"
    text = requests.get(url).content.decode('utf8')
    with io.open('language-never-random.txt', 'w', encoding='utf8') as fout:
        fout.write(text)


In [ ]:
# Tokenize the text.
tokenized_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in sent_tokenize(text)]

In [ ]:
tokenized_text[0]

['language',
 'is',
 'never',
 ',',
 'ever',
 ',',
 'ever',
 ',',
 'random',
 'adam',
 'kilgarriff',
 'abstract',
 'language',
 'users',
 'never',
 'choose',
 'words',
 'randomly',
 ',',
 'and',
 'language',
 'is',
 'essentially',
 'non-random',
 '.']

In [ ]:
print(text[:500])

                       Language is never, ever, ever, random

                                                               ADAM KILGARRIFF




Abstract
Language users never choose words randomly, and language is essentially
non-random. Statistical hypothesis testing uses a null hypothesis, which
posits randomness. Hence, when we look at linguistic phenomena in cor-
pora, the null hypothesis will never be true. Moreover, where there is enough
data, we shall (almost) always be able to establish 


In [ ]:
# Preprocess the tokenized text for 3-grams language modelling
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

**Training an N-gram Model**

Corpus yang sudah ready setelah langkah diatas dapat digunakan untuk training model bahasa. Sebagai contoh sederhana, kita akan melatih model Maximum Likelihood Estimator (MLE). Pelatihan model ini memerlukan spesifikasi n-grams. Contoh dibawah menggunakan trigram (n=3).

In [ ]:
from nltk.lm import MLE
model = MLE(n) # Lets train a 3-grams model, previously we set n=3

Inisialiasi vocabulary dari model

In [ ]:
len(model.vocab)

0

vocabulary model ini akan terisi setelah melakukan pelatihan model

In [ ]:
model.fit(train_data, padded_sents)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 1391 items>


In [ ]:
len(model.vocab)

1391

Vocabulary model dapat membantu mendeteksi kata-kata yang tidak pernah muncul selama proses training.

In [ ]:
print(model.vocab.lookup(tokenized_text[0]))

('language', 'is', 'never', ',', 'ever', ',', 'ever', ',', 'random', 'adam', 'kilgarriff', 'abstract', 'language', 'users', 'never', 'choose', 'words', 'randomly', ',', 'and', 'language', 'is', 'essentially', 'non-random', '.')


 kata-kata yang tidak pernah muncul dalam vocabulary model akan diganti dengan `<UNK>`

In [ ]:
# If we lookup the vocab on unseen sentences not from the training data, 
# it automatically replace words not in the vocabulary with `<UNK>`.
print(model.vocab.lookup('language is never random lah .'.split()))

('language', 'is', 'never', 'random', '<UNK>', '.')


Dalam beberapa kasus,kata-kata yang jarang muncul pada saat training dapat kita abaikan untuk mendapatkan informasi yang penting dari model. Perintah untuk mengabaikan kata yang jarang muncul yaitu `unk_cutoff`.

Catatan: Untuk model ngram lainnya, lihat objek ini dari `nltk.lm.models`:

- `Lidstone`: Provides Lidstone-smoothed scores.
- `Laplace`: Implements Laplace (add one) smoothing.
- `InterpolatedLanguageModel`: Logic common to all interpolated language model (Chen & Goodman 1995).
- `WittenBellInterpolated`: Interpolated version of Witten-Bell smoothing.

**Using the N-gram Language Model**

Menghitung ngram pada sebuah corpus.

In [ ]:
print(model.counts)

<NgramCounter with 3 ngram orders and 19611 ngrams>


perhitungan banyak kata "language" pada corpus


In [ ]:
model.counts['language'] # i.e. Count('language')

25

perhitungan banyak kata "is" setelah kata "language"

In [ ]:
model.counts[['language']]['is'] # i.e. Count('is'|'language')

11

perhitungan banyak kata "never" setelah kata "language is"





In [ ]:
model.counts[['language', 'is']]['never'] # i.e. Count('never'|'language is')

7

Dan seterusnya. Namun, tujuan sebenarnya dari melatih language modeling adalah untuk menilai seberapa besar kemungkinan kata-kata dalam konteks tertentu.

Dengan MLE ini, model mengembalikan frekuensi relatif item sebagai skornya

Probability muncul kata "language" pada model

In [ ]:
model.score('language') # P('language')

0.003691671588895452

Probability muncul kata "is" setelah kata "language" pada model

In [ ]:
model.score('is', 'language'.split())  # P('is'|'language')

0.44

Probability muncul kata "never" setelah kata "language is" pada model

In [ ]:
model.score('never', 'language is'.split())  # P('never'|'language is')

0.6363636363636364

Melakukan pengecekan apakah simbol "`<UNK>`" merupakan kata "lah" "leh" "lor"

In [ ]:
model.score("<UNK>") == model.score("lah")

True

In [ ]:
model.score("<UNK>") == model.score("leh")

True

In [ ]:
model.score("<UNK>") == model.score("lor")

True

Untuk menghindari nilai 0 atau kecil menggunakan logaritma "logscore"

In [ ]:
model.logscore("never", "language is".split())

-0.6520766965796932

**Generation using N-gram Language Model**

Adapun fitur generating text pada n-gram salah satu fitur yang dapat model ini adalah kita tidak perlu melacak keseluruhan kalimat untuk memprediksi kata berikutnya, kita hanya perlu melihat kembali n-1 token.

In [ ]:
print(model.generate(20, random_seed=7))

['and', 'hence', ',', 'when', 'we', 'look', 'at', 'linguistic', 'phenom-', 'ena', 'in', 'corpora', ',', '216⫺223', '.', '</s>', '</s>', '</s>', '</s>', '</s>']


Melakukan pembersihan sebelum generated token seperti yang dilakukan manusia

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

Model diminta untuk menghasilkan data keluaran(output) berdasarkan pelatihan yang dilakukan

In [ ]:
generate_sent(model, 20, random_seed=7)

'and hence, when we look at linguistic phenom- ena in corpora , 216⫺223.'

In [ ]:
print(model.generate(28, random_seed=0))

['the', 'trouble', 'with', 'quantitative', 'studies', '.', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']


In [ ]:
generate_sent(model, 28, random_seed=0)

'the trouble with quantitative studies.'

In [ ]:
generate_sent(model, 20, random_seed=1)

'237⫺246.'

In [ ]:
generate_sent(model, 20, random_seed=30)

'hypothesis test- ing has been used in the joint conference on empirical methods in nlp and very large corpora although'

In [ ]:
generate_sent(model, 20, random_seed=42)

'more common or more salient in the last paragraph is ‘ indistinguishable ’.'

### **Sumber:**

1.   https://www.kaggle.com/rtatman/tutorial-getting-n-grams
2.   https://medium.com/swlh/language-modelling-with-nltk-20eac7e70853

